In [1]:
#libraries
import pandas as pd
import numpy as np

#Get the CSV file.
flnm = "sinkhole_sample1.csv"
filestr = "dataset/" + flnm

In [2]:
# Take raw data to the Raw_Data dataset.
Raw_Data = pd.read_csv(filestr , index_col = "No.")

# Converting Raw_Data to numpy array
np_Raw_Data = np.array(Raw_Data)

In [3]:
# Sorting data on 0 axis.(0 axis is the time values.)
# The columns 0: Time, 1: Source, 2:Destination, 3:Protocol, 4:Packet Length, 5:Info
np_Raw_Data = np_Raw_Data[np.argsort(np_Raw_Data[:, 0])]


In [4]:
# packetDurations list is used for calculating packet durations. With the while loop below, 
# we substracted two values np_Raw_Data[n][0] - np_Raw_Data[n - 1][0] and appended to the 
# packetDurations list.
packetDurations = []
counter = 0
while counter < len(np_Raw_Data):
    duration = 0
    if counter != 0 and counter + 1 <  len(np_Raw_Data):
        duration = np.float32(np_Raw_Data[counter][0])-np.float32(np_Raw_Data[counter - 1][0])
    packetDurations.append(duration) 
    counter +=  1

In [5]:

# Delete the first row of packetDurations
packetDurations = np.delete(packetDurations, 0, axis = 0)

# Delete the last row of np_Raw_Data
np_Raw_Data = np.delete(np_Raw_Data,len(np_Raw_Data)-1,axis = 0)

In [6]:
# Add  packetDurations column to the np_Raw_Data as 1st column.
# The columns 0: Time, 1:Packet Durations, 2: Source, 3:Destination, 4:Protocol, 5:Packet Length, 6:Info
np_Raw_Data = np.insert(np_Raw_Data, 1, packetDurations, axis = 1)

# source_unique_array variable contains unique values of source IP addresses.
source_unique_array = np.unique(np.array(Raw_Data.iloc[:,1:2].astype(str)))

# destination_unique_array variable contains unique values of destination IP addresses.
destination_unique_array = np.unique(np.array(Raw_Data.iloc[:,2:3].astype(str)))

# info_unique_array variable contains unique values of info colunm.
info_unique_array = np.unique(np.array(Raw_Data.iloc[:,5:6]))

# protocol_unique_array variable contains unique values of protocol column.
protocol_unique_array = np.unique(np.array(Raw_Data.iloc[:,3:4]))

# all_ip_addresses variable contains unique values of all IP addresses.
all_ip_addresses = np.concatenate((source_unique_array,destination_unique_array))
all_ip_addresses = np.unique(all_ip_addresses)

# ip_dict dictionary will hold the IPV6:IP Number key value pairs.
# example fe80::c30c:0:0:1 : 0
ip_dict = {}

In [7]:
# Use sklearn labelEncoder to give numbers to the ip addresses.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
lb_all_ip_addresses = le.fit_transform(all_ip_addresses)

# with the for loop add Ip address key value pairs.
cnt = 0
for x in all_ip_addresses:
    ip_dict[x] = lb_all_ip_addresses[cnt]
    cnt  += 1

# Sorting data on 0 axis.(0 axis is the time values.)
np_Raw_Data = np_Raw_Data[np.argsort(np_Raw_Data[:, 0])]

# duration variable is the last second of time column.
duration = np.floor(np.float32(np_Raw_Data[-1][0]))

In [8]:
# Variables that are used for calculating values.
counter = 0
currentSecond = 60.0
packetcount = {}
TotalPacketDuration = {}
TotalPacketLenght = {}
src_count = {}
dst_count = {}
src_duration = {}
dst_duration = {}
src_packet_lenght_sum = {}
dst_packet_lenght_sum = {}
DioCount = {}
DisCount = {}
DaoCount = {}
OtherMsg = {}
frame = []

In [9]:
row = pd.DataFrame(columns = ['second','src', 'dst','packetcount','src_ratio', 'dst_ratio','src_duration_ratio', 'dst_duration_ratio','TotalPacketDuration','TotalPacketLenght','src_packet_ratio','dst_packet_ratio','DioCount','DisCount','DaoCount','OtherMsg'])


In [10]:
while counter < duration:  
    
    # one_second_frame variable holds rows for 1 second duration.
    one_second_frame = np_Raw_Data[np.where(np.logical_and(np_Raw_Data[:, 0] >= currentSecond, np_Raw_Data[:, 0] <= currentSecond + 1.0))]
    
    # if there is data in the one_second_frame, make calculations.
    if one_second_frame.size > 1:
        # clear all variables.
        packetcount.clear()
        TotalPacketDuration.clear()
        TotalPacketLenght.clear()
        DioCount.clear()
        DisCount.clear()
        DaoCount.clear()
        src_duration.clear()
        dst_duration.clear()
        totalpackets = 0
        frame_packet_length_sum = 0
        total_duration = 0.0
        src_packet_lenght_sum.clear()
        dst_packet_lenght_sum.clear()
        src_count.clear()
        dst_count.clear()
        
        # looping in each one_second_frame row
        for packet in one_second_frame:
            # IEEE 802.15.4 protocols or Ack messages do not have no IP addresses and they are null. Do not process this data.
            if not pd.isnull(packet[2]):
                # src_dst variable is string and it holds the value of source and destination value like fe80::c30c:0:0:3-fe80::c30c:0:0:1
                src = packet[2]
                dst = packet[3]
                src_dst  =  src + "-" + dst
                
                # packetcount dictionary holds the "src_dst : count" key value pairs. (How many packey counts do we have source to destination?)
                packetcount[src_dst]  =  1 if src_dst not in packetcount else packetcount[src_dst] + 1
                
                # TotalPacketDuration dictionary holds the "src_dst : duration" key value pairs. (It is the sum of all packet durations from source to destination in the 1-second frame.)
                TotalPacketDuration[src_dst] = packet[1] if src_dst not in TotalPacketDuration else TotalPacketDuration[src_dst] + packet[1]
                
                # TotalPacketLenght dictionary holds the "src_dst : length" key value pairs. (It is the sum of all packet lengths from source to destination in the 1-second frame.)
                TotalPacketLenght[src_dst] = packet[5] if src_dst not in TotalPacketLenght else TotalPacketLenght[src_dst] + packet[5]
                
                # src_count dictionary holds the "source : count" key value pairs. (How many source IP adress has in 1 second frame?)
                src_count[src] = 1 if src not in src_count else src_count[src] + 1
                
                # dst_count dictionary holds the "destination : count" key value pairs. (How many destination IP adress has in 1 second frame?)
                dst_count[dst] = 1 if dst not in dst_count else dst_count[dst] + 1
                
                # src_duration dictionary holds the "source : duration" key value pairs. (What is the duration of source IP address in 1 second frame?)
                src_duration[src] = packet[1] if src not in src_duration else src_duration[src] + packet[1]
                
                # dst_duration dictionary holds the "destination : duration" key value pairs. (What is the duration of destination IP address in 1 second frame?)
                dst_duration[dst] = packet[1] if dst not in dst_duration else dst_duration[dst] + packet[1]
                
                # total_duration will be used for calculating ratios in a one second frame
                total_duration += packet[1]
                
                # src_packet_lenght_sum dictionary holds "source : source_packet_length_sum" key value pairs.
                src_packet_lenght_sum[src] = packet[5] if src not in src_packet_lenght_sum else src_packet_lenght_sum[src] + packet[5]
                
                 # dst_packet_lenght_sum dictionary holds "destination : destination_packet_length_sum" key value pairs.
                dst_packet_lenght_sum[dst] = packet[5] if dst not in dst_packet_lenght_sum else dst_packet_lenght_sum[dst] + packet[5]
                
                # frame_packet_length_sum will be used for calculating ratios in a one second frame
                frame_packet_length_sum +=  packet[5]
                
                # totalpackets will be used for calculating ratios in a one second frame
                totalpackets +=  1
                
                # DIO, DIS, DAO messages counts.
                if packet[6]=="RPL Control (DODAG Information Object)":
                    DioCount[src_dst] = 1 if src_dst not in DioCount else DioCount[src_dst] + 1
                if packet[6]=="RPL Control (DODAG Information Solicitation)":
                    DisCount[src_dst] = 1 if src_dst not in DisCount else DisCount[src_dst] + 1
                if packet[6]=="RPL Control (Destination Advertisement Object)":
                    DaoCount[src_dst] = 1 if src_dst not in DaoCount else DaoCount[src_dst] + 1
                if ((packet[6]!="RPL Control (Destination Advertisement Object)") and (packet[6]!="RPL Control (DODAG Information Object)") and (packet[6]!="RPL Control (Destination Advertisement Object)")) :
                    OtherMsg[src_dst] = 1 if src_dst not in OtherMsg else OtherMsg[src_dst] + 1
       
        # this for loop calculates the ratios.
        for i in packetcount:
            if not i in DioCount:
                arr_diocount = 0
            else:
                arr_diocount = DioCount[i]
            if not i in DisCount:
                arr_discount = 0
            else:
                arr_discount = DisCount[i]
            if not i in DaoCount:
                arr_daocount = 0
            else:
                arr_daocount = DaoCount[i]
            if not i in OtherMsg:
                arr_orhermsg = 0
            else:
                arr_orhermsg = OtherMsg[i]
                
            # Splitting source and destination
            x = i.split("-")
            sourcee = x[0]
            destinatt = x[1]
            
            # calculating the source ratio in 1 second frame
            src_ratio = src_count[sourcee]/totalpackets
            
            # calculating the destination ratio in 1 second frame
            if totalpackets==0:
                dst_ratio=0
            else:
                
                dst_ratio = dst_count[destinatt]/totalpackets
            
            
            if total_duration==0:
                src_duration_ratio=0
                dst_duration_ratio=0
            else:
                
            
                # calculating the source duration ratio in 1 second frame
                src_duration_ratio = src_duration[sourcee]/total_duration

                # calculating the destination duration ratio in 1 second frame
                dst_duration_ratio = dst_duration[destinatt]/total_duration

            # calculating the source packet length ratio in 1 second frame
            src_packet_ratio = src_packet_lenght_sum[sourcee]/frame_packet_length_sum
            
            # calculating the destination pacekt length ratio in 1 second frame
            dst_packet_ratio = dst_packet_lenght_sum[destinatt]/frame_packet_length_sum
            
            #establishing an array for adding the calculations to the row of row dataframe.
            # the columns are: 
            # 'second',
            # 'src', 
            # 'dst',
            # 'packetcount',
            # 'src_ratio', 
            # 'dst_ratio',
            # 'src_duration_ratio', 
            # 'dst_duration_ratio',
            # 'TotalPacketDuration',
            # 'TotalPacketLenght',
            # 'src_packet_ratio',
            # 'dst_packet_ratio',
            # 'DioCount',
            # 'DisCount',
            # 'DaoCount',
            # 'OtherMsg'
            array = np.array([
                np.single(currentSecond),
                ip_dict[sourcee],
                ip_dict[destinatt],
                int(packetcount[i]),
                np.single(src_ratio),
                np.single(dst_ratio),
                np.single(src_duration_ratio),
                np.single(dst_duration_ratio),
                TotalPacketDuration[i],
                TotalPacketLenght[i],
                np.single(src_packet_ratio),
                np.single(dst_packet_ratio),
                arr_diocount,
                arr_discount,
                arr_daocount,
                arr_orhermsg], dtype="object")
            a_series = pd.Series(array, index = row.columns)            
            row = pd.concat([row, a_series.to_frame().T], ignore_index=True)
            
    # increase second 1
    currentSecond += 1.0
    
    # increase counter 1
    counter +=  1
    
    # print data for observing the process.
    print(str(counter) + " of " + str(duration) + " of process is ok!!!")

1 of 3443.0 of process is ok!!!
2 of 3443.0 of process is ok!!!
3 of 3443.0 of process is ok!!!
4 of 3443.0 of process is ok!!!
5 of 3443.0 of process is ok!!!
6 of 3443.0 of process is ok!!!
7 of 3443.0 of process is ok!!!
8 of 3443.0 of process is ok!!!
9 of 3443.0 of process is ok!!!
10 of 3443.0 of process is ok!!!
11 of 3443.0 of process is ok!!!
12 of 3443.0 of process is ok!!!
13 of 3443.0 of process is ok!!!
14 of 3443.0 of process is ok!!!
15 of 3443.0 of process is ok!!!
16 of 3443.0 of process is ok!!!
17 of 3443.0 of process is ok!!!
18 of 3443.0 of process is ok!!!
19 of 3443.0 of process is ok!!!
20 of 3443.0 of process is ok!!!
21 of 3443.0 of process is ok!!!
22 of 3443.0 of process is ok!!!
23 of 3443.0 of process is ok!!!
24 of 3443.0 of process is ok!!!
25 of 3443.0 of process is ok!!!
26 of 3443.0 of process is ok!!!
27 of 3443.0 of process is ok!!!
28 of 3443.0 of process is ok!!!
29 of 3443.0 of process is ok!!!
30 of 3443.0 of process is ok!!!
31 of 3443.0 of pro

282 of 3443.0 of process is ok!!!
283 of 3443.0 of process is ok!!!
284 of 3443.0 of process is ok!!!
285 of 3443.0 of process is ok!!!
286 of 3443.0 of process is ok!!!
287 of 3443.0 of process is ok!!!
288 of 3443.0 of process is ok!!!
289 of 3443.0 of process is ok!!!
290 of 3443.0 of process is ok!!!
291 of 3443.0 of process is ok!!!
292 of 3443.0 of process is ok!!!
293 of 3443.0 of process is ok!!!
294 of 3443.0 of process is ok!!!
295 of 3443.0 of process is ok!!!
296 of 3443.0 of process is ok!!!
297 of 3443.0 of process is ok!!!
298 of 3443.0 of process is ok!!!
299 of 3443.0 of process is ok!!!
300 of 3443.0 of process is ok!!!
301 of 3443.0 of process is ok!!!
302 of 3443.0 of process is ok!!!
303 of 3443.0 of process is ok!!!
304 of 3443.0 of process is ok!!!
305 of 3443.0 of process is ok!!!
306 of 3443.0 of process is ok!!!
307 of 3443.0 of process is ok!!!
308 of 3443.0 of process is ok!!!
309 of 3443.0 of process is ok!!!
310 of 3443.0 of process is ok!!!
311 of 3443.0 

612 of 3443.0 of process is ok!!!
613 of 3443.0 of process is ok!!!
614 of 3443.0 of process is ok!!!
615 of 3443.0 of process is ok!!!
616 of 3443.0 of process is ok!!!
617 of 3443.0 of process is ok!!!
618 of 3443.0 of process is ok!!!
619 of 3443.0 of process is ok!!!
620 of 3443.0 of process is ok!!!
621 of 3443.0 of process is ok!!!
622 of 3443.0 of process is ok!!!
623 of 3443.0 of process is ok!!!
624 of 3443.0 of process is ok!!!
625 of 3443.0 of process is ok!!!
626 of 3443.0 of process is ok!!!
627 of 3443.0 of process is ok!!!
628 of 3443.0 of process is ok!!!
629 of 3443.0 of process is ok!!!
630 of 3443.0 of process is ok!!!
631 of 3443.0 of process is ok!!!
632 of 3443.0 of process is ok!!!
633 of 3443.0 of process is ok!!!
634 of 3443.0 of process is ok!!!
635 of 3443.0 of process is ok!!!
636 of 3443.0 of process is ok!!!
637 of 3443.0 of process is ok!!!
638 of 3443.0 of process is ok!!!
639 of 3443.0 of process is ok!!!
640 of 3443.0 of process is ok!!!
641 of 3443.0 

885 of 3443.0 of process is ok!!!
886 of 3443.0 of process is ok!!!
887 of 3443.0 of process is ok!!!
888 of 3443.0 of process is ok!!!
889 of 3443.0 of process is ok!!!
890 of 3443.0 of process is ok!!!
891 of 3443.0 of process is ok!!!
892 of 3443.0 of process is ok!!!
893 of 3443.0 of process is ok!!!
894 of 3443.0 of process is ok!!!
895 of 3443.0 of process is ok!!!
896 of 3443.0 of process is ok!!!
897 of 3443.0 of process is ok!!!
898 of 3443.0 of process is ok!!!
899 of 3443.0 of process is ok!!!
900 of 3443.0 of process is ok!!!
901 of 3443.0 of process is ok!!!
902 of 3443.0 of process is ok!!!
903 of 3443.0 of process is ok!!!
904 of 3443.0 of process is ok!!!
905 of 3443.0 of process is ok!!!
906 of 3443.0 of process is ok!!!
907 of 3443.0 of process is ok!!!
908 of 3443.0 of process is ok!!!
909 of 3443.0 of process is ok!!!
910 of 3443.0 of process is ok!!!
911 of 3443.0 of process is ok!!!
912 of 3443.0 of process is ok!!!
913 of 3443.0 of process is ok!!!
914 of 3443.0 

1259 of 3443.0 of process is ok!!!
1260 of 3443.0 of process is ok!!!
1261 of 3443.0 of process is ok!!!
1262 of 3443.0 of process is ok!!!
1263 of 3443.0 of process is ok!!!
1264 of 3443.0 of process is ok!!!
1265 of 3443.0 of process is ok!!!
1266 of 3443.0 of process is ok!!!
1267 of 3443.0 of process is ok!!!
1268 of 3443.0 of process is ok!!!
1269 of 3443.0 of process is ok!!!
1270 of 3443.0 of process is ok!!!
1271 of 3443.0 of process is ok!!!
1272 of 3443.0 of process is ok!!!
1273 of 3443.0 of process is ok!!!
1274 of 3443.0 of process is ok!!!
1275 of 3443.0 of process is ok!!!
1276 of 3443.0 of process is ok!!!
1277 of 3443.0 of process is ok!!!
1278 of 3443.0 of process is ok!!!
1279 of 3443.0 of process is ok!!!
1280 of 3443.0 of process is ok!!!
1281 of 3443.0 of process is ok!!!
1282 of 3443.0 of process is ok!!!
1283 of 3443.0 of process is ok!!!
1284 of 3443.0 of process is ok!!!
1285 of 3443.0 of process is ok!!!
1286 of 3443.0 of process is ok!!!
1287 of 3443.0 of pr

...

2107 of 3443.0 of process is ok!!!
2108 of 3443.0 of process is ok!!!
2109 of 3443.0 of process is ok!!!
2110 of 3443.0 of process is ok!!!
2111 of 3443.0 of process is ok!!!
2112 of 3443.0 of process is ok!!!
2113 of 3443.0 of process is ok!!!
2114 of 3443.0 of process is ok!!!
2115 of 3443.0 of process is ok!!!
2116 of 3443.0 of process is ok!!!
2117 of 3443.0 of process is ok!!!
2118 of 3443.0 of process is ok!!!
2119 of 3443.0 of process is ok!!!
2120 of 3443.0 of process is ok!!!
2121 of 3443.0 of process is ok!!!
2122 of 3443.0 of process is ok!!!
2123 of 3443.0 of process is ok!!!
2124 of 3443.0 of process is ok!!!
2125 of 3443.0 of process is ok!!!
2126 of 3443.0 of process is ok!!!
2127 of 3443.0 of process is ok!!!
2128 of 3443.0 of process is ok!!!
2129 of 3443.0 of process is ok!!!
2130 of 3443.0 of process is ok!!!
2131 of 3443.0 of process is ok!!!
2132 of 3443.0 of process is ok!!!
2133 of 3443.0 of process is ok!!!
2134 of 3443.0 of process is ok!!!
2135 of 3443.0 of pr

2468 of 3443.0 of process is ok!!!
2469 of 3443.0 of process is ok!!!
2470 of 3443.0 of process is ok!!!
2471 of 3443.0 of process is ok!!!
2472 of 3443.0 of process is ok!!!
2473 of 3443.0 of process is ok!!!
2474 of 3443.0 of process is ok!!!
2475 of 3443.0 of process is ok!!!
2476 of 3443.0 of process is ok!!!
2477 of 3443.0 of process is ok!!!
2478 of 3443.0 of process is ok!!!
2479 of 3443.0 of process is ok!!!
2480 of 3443.0 of process is ok!!!
2481 of 3443.0 of process is ok!!!
2482 of 3443.0 of process is ok!!!
2483 of 3443.0 of process is ok!!!
2484 of 3443.0 of process is ok!!!
2485 of 3443.0 of process is ok!!!
2486 of 3443.0 of process is ok!!!
2487 of 3443.0 of process is ok!!!
2488 of 3443.0 of process is ok!!!
2489 of 3443.0 of process is ok!!!
2490 of 3443.0 of process is ok!!!
2491 of 3443.0 of process is ok!!!
2492 of 3443.0 of process is ok!!!
2493 of 3443.0 of process is ok!!!
2494 of 3443.0 of process is ok!!!
2495 of 3443.0 of process is ok!!!
2496 of 3443.0 of pr

2776 of 3443.0 of process is ok!!!
2777 of 3443.0 of process is ok!!!
2778 of 3443.0 of process is ok!!!
2779 of 3443.0 of process is ok!!!
2780 of 3443.0 of process is ok!!!
2781 of 3443.0 of process is ok!!!
2782 of 3443.0 of process is ok!!!
2783 of 3443.0 of process is ok!!!
2784 of 3443.0 of process is ok!!!
2785 of 3443.0 of process is ok!!!
2786 of 3443.0 of process is ok!!!
2787 of 3443.0 of process is ok!!!
2788 of 3443.0 of process is ok!!!
2789 of 3443.0 of process is ok!!!
2790 of 3443.0 of process is ok!!!
2791 of 3443.0 of process is ok!!!
2792 of 3443.0 of process is ok!!!
2793 of 3443.0 of process is ok!!!
2794 of 3443.0 of process is ok!!!
2795 of 3443.0 of process is ok!!!
2796 of 3443.0 of process is ok!!!
2797 of 3443.0 of process is ok!!!
2798 of 3443.0 of process is ok!!!
2799 of 3443.0 of process is ok!!!
2800 of 3443.0 of process is ok!!!
2801 of 3443.0 of process is ok!!!
2802 of 3443.0 of process is ok!!!
2803 of 3443.0 of process is ok!!!
2804 of 3443.0 of pr

3130 of 3443.0 of process is ok!!!
3131 of 3443.0 of process is ok!!!
3132 of 3443.0 of process is ok!!!
3133 of 3443.0 of process is ok!!!
3134 of 3443.0 of process is ok!!!
3135 of 3443.0 of process is ok!!!
3136 of 3443.0 of process is ok!!!
3137 of 3443.0 of process is ok!!!
3138 of 3443.0 of process is ok!!!
3139 of 3443.0 of process is ok!!!
3140 of 3443.0 of process is ok!!!
3141 of 3443.0 of process is ok!!!
3142 of 3443.0 of process is ok!!!
3143 of 3443.0 of process is ok!!!
3144 of 3443.0 of process is ok!!!
3145 of 3443.0 of process is ok!!!
3146 of 3443.0 of process is ok!!!
3147 of 3443.0 of process is ok!!!
3148 of 3443.0 of process is ok!!!
3149 of 3443.0 of process is ok!!!
3150 of 3443.0 of process is ok!!!
3151 of 3443.0 of process is ok!!!
3152 of 3443.0 of process is ok!!!
3153 of 3443.0 of process is ok!!!
3154 of 3443.0 of process is ok!!!
3155 of 3443.0 of process is ok!!!
3156 of 3443.0 of process is ok!!!
3157 of 3443.0 of process is ok!!!
3158 of 3443.0 of pr

3407 of 3443.0 of process is ok!!!
3408 of 3443.0 of process is ok!!!
3409 of 3443.0 of process is ok!!!
3410 of 3443.0 of process is ok!!!
3411 of 3443.0 of process is ok!!!
3412 of 3443.0 of process is ok!!!
3413 of 3443.0 of process is ok!!!
3414 of 3443.0 of process is ok!!!
3415 of 3443.0 of process is ok!!!
3416 of 3443.0 of process is ok!!!
3417 of 3443.0 of process is ok!!!
3418 of 3443.0 of process is ok!!!
3419 of 3443.0 of process is ok!!!
3420 of 3443.0 of process is ok!!!
3421 of 3443.0 of process is ok!!!
3422 of 3443.0 of process is ok!!!
3423 of 3443.0 of process is ok!!!
3424 of 3443.0 of process is ok!!!
3425 of 3443.0 of process is ok!!!
3426 of 3443.0 of process is ok!!!
3427 of 3443.0 of process is ok!!!
3428 of 3443.0 of process is ok!!!
3429 of 3443.0 of process is ok!!!
3430 of 3443.0 of process is ok!!!
3431 of 3443.0 of process is ok!!!
3432 of 3443.0 of process is ok!!!
3433 of 3443.0 of process is ok!!!
3434 of 3443.0 of process is ok!!!
3435 of 3443.0 of pr

In [11]:
import os

if not os.path.exists('Results'):
    os.makedirs('Results')


In [12]:
#Save as CSV file
row.to_csv('Results/sinkhole.csv', index=False, sep=',')
